In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
import sys
sys.path.append('./ui/')
from UiAboutSystem import Ui_AboutSystem
from UiContactUs import Ui_ContactUs
from UiMainwindow import Ui_MainWindow
import UiAboutSystem
import UiContactUs
import UiMainwindow

In [1]:
from PyQt5 import QtWidgets
import pyqtgraph as pg
from typing import Any,Dict
import random,sys
 
class RotateAxisItem(pg.AxisItem):
    def drawPicture(self, p, axisSpec, tickSpecs, textSpecs):
        p.setRenderHint(p.Antialiasing,False)
        p.setRenderHint(p.TextAntialiasing,True)
 
        ## draw long line along axis
        pen,p1,p2 = axisSpec
        p.setPen(pen)
        p.drawLine(p1,p2)
        p.translate(0.5,0)  ## resolves some damn pixel ambiguity
 
        for pen,p1,p2 in tickSpecs:
            p.setPen(pen)
            p.drawLine(p1,p2)
 
        p.setPen(self.pen())
        for rect,flags,text in textSpecs:
            # this is the important part
            p.save()
            p.translate(rect.x(),rect.y())
            p.rotate(-30)
            p.drawText(-rect.width(),rect.height(),rect.width(),rect.height(),flags,text)
            # restoring the painter is *required*!!!
            p.restore()
 
class LineShowManager(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
 
        self.y_datas = []
 
        self.init_ui()
        pass
 
    def init_ui(self) -> None:
 
        pg.setConfigOptions(leftButtonPan=False, antialias=True)
        pg.setConfigOption('background', 'w')
        pg.setConfigOption('foreground', 'k')
 
        xax = RotateAxisItem(orientation='bottom')
        xax.setHeight(h=80)
        self.pw = pg.PlotWidget(axisItems={'bottom': xax})
        self.pw.setLabel("left", "值")
        self.pw.setLabel("bottom", "时间")
        self.label = pg.TextItem()
        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.pw)
        self.setLayout(layout)
        pass
 
    def plot_data(self, data: Dict[str, Any]):
        self.pw.clear()
        self.pw.addLegend()
        # xTick x  多条曲线共用x轴数据
        xTick = [data['xTick']]
        x = data['x']
        y_list = data['y_list']
        y_names = data['y_names']
        y_min = data['y_min']
        y_max = data['y_max']
        self.y_datas = y_list
        self.x_data = xTick
        self.y_names = y_names
 
        xax = self.pw.getAxis('bottom')
        xax.setTicks(xTick)
 
        for i in range(len(y_names)):
            self.pw.plot(x, y_list[i], connect='finite', pen=pg.mkPen(
                {'color': (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), 'width': 4}),
                         name=y_names[i])
        self.vLine = pg.InfiniteLine(angle=90, movable=False)
        self.hLine = pg.InfiniteLine(angle=0, movable=False)
        self.pw.addItem(self.vLine, ignoreBounds=True)
        self.pw.addItem(self.hLine, ignoreBounds=True)
        self.pw.addItem(self.label, ignoreBounds=True)
        self.pw.setRange(xRange=[0,len(x)],yRange=[y_min,y_max])
        self.vb = self.pw.getViewBox()
        self.proxy = pg.SignalProxy(self.pw.scene().sigMouseMoved, rateLimit=60, slot=self.mouseMoved)
        pass
 
    def mouseMoved(self, evt):
        pos = evt[0]
        if len(self.y_datas) <= 0:
            return
        if self.pw.sceneBoundingRect().contains(pos):
            mousePoint = self.vb.mapSceneToView(pos)
            index = int(mousePoint.x())
            pos_y = int(mousePoint.y())
 
            if index >= 0 and index < len(self.y_datas[0]):
                # Here we have obtained the mouse x point
                # So lets use a for loop for each curve to set the Pos
                time_str = self.x_data[0][index][1]
                # time_str = ''
                data_str = ''
                for m in range(len(self.y_datas)):
                    data_str += '<p style="color:black;">' + self.y_names[m] + ':' + str(
                        self.y_datas[m][index]) + '</p>'
                html_str = '<p style="color:black;">' + time_str + '</p>' + data_str
                self.label.setHtml(html_str)
                self.label.setPos(mousePoint.x(), mousePoint.y())
            self.vLine.setPos(mousePoint.x())
            self.hLine.setPos(mousePoint.y())
        pass
 
    def showEvent(self, QShowEvent):
        self.showMaximized()
 
if __name__ == '__main__':
    temp_map = {}
    temp_map['xTick'] = [(0,'2021-01-01'),
                         (1,'2021-02-0'),
                         (2,'2021-03-01'),
                         (3,'2021-04-01'),
                         (4,'2021-05-01'),
                         (5,'2021-06-01'),
                         (6,'2021-07-01'),
                         ]
    temp_map['x'] = [0,1,2,3,4,5,6]
    temp_map['y_list'] = [[1,8,0,5,1,-6,3],[-9,12,6,0,8,-2,8],[-7,19,16,-9,10,-6,8]]
    temp_map['y_names'] = ['线条1','线条2','线条3']
    temp_map['y_min'] = -9
    temp_map['y_max'] = 19
 
    app = QtWidgets.QApplication(sys.argv)
    temp_widget = LineShowManager()
    temp_widget.show()
    temp_widget.plot_data(temp_map)
    sys.exit(app.exec_())
 
 

C:\Users\qwer1\AppData\Local\Temp\ipykernel_34412\3561725260.py:27: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  p.drawText(-rect.width(),rect.height(),rect.width(),rect.height(),flags,text)
C:\Users\qwer1\AppData\Local\Temp\ipykernel_34412\3561725260.py:27: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  p.drawText(-rect.width(),rect.height(),rect.width(),rect.height(),flags,text)
C:\Users\qwer1\AppData\Local\Temp\ipykernel_34412\3561725260.py:27: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  p.drawText(-rect.width(),rect.height(),rect.width(),rect.height(),flags,text)
C:\Users\qwer1\AppData\Local\Temp\ipy

SystemExit: 0

c:\Users\qwer1\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
'''
C++ FILES: 
Descripttion: 
version: 
Author: TheiiKu
Date: 2023-11-23 15:59:27
LastEditors: TheiiKu
LastEditTime: 2023-11-23 17:52:17
afterAnnotation: CRATED IN JLU SENSOR LAB
'''
class ContactPage(QtWidgets.QMainWindow ,  Ui_ContactUs):
    def __init__(self):
        super().__init__()
        self.setupUi(self)

        
    def openGithub(self):

        import webbrowser
        webbrowser.open("https://github.com/ToreyGu")
        pass
    def openLabweb(self):
        import webbrowser
        webbrowser.open("https://sensors.jlu.edu.cn/")
        pass

In [3]:
import wmi

import base64
import hashlib
from pyDes import *
import time

LICDAY = 10
m_wmi = wmi.WMI()
#获取唯一硬件码
def get_CPU_Serial():

    m_wmi = wmi.WMI()
    cpu_info = m_wmi.Win32_Processor()
    if len(cpu_info) > 0:
        serial_number = cpu_info[0].ProcessorId
        print(serial_number)
    return serial_number

def get_Mac_adress():
    mac_buffer =''
    m_wmi = wmi.WMI()
    for network in m_wmi.Win32_NetworkAdapterConfiguration():
        mac_address = network.MacAddress
        if mac_address != None:
            mac_buffer = mac_buffer + mac_address +','
    print(mac_buffer)
    return mac_buffer

def get_Board_Serial():
    m_wmi = wmi.WMI()
    disk_info = m_wmi.Win32_PhysicalMedia()
    if len(disk_info) > 0:
        serial_number = disk_info[0].SerialNumber.strip()
    return serial_number
 
CPU_Serial =  get_CPU_Serial()

Mac_adress = get_Mac_adress()

Board_Serial = get_Board_Serial()

#计算机的唯一编码
ComputerInfo = CPU_Serial + Mac_adress + Board_Serial
ComputerInfoByte = ComputerInfo.encode('UTF-8')

#计算机唯一等长机器码
MachineCode   =  hashlib.md5(ComputerInfoByte ).hexdigest()



import json
data = { 'MachineCode' : MachineCode, 'StartTime' : time.time(), 'FinTime' : time.time() + 1000*60*60*24*LICDAY , 'Software' : 'Gyrflacon'} 
time.strftime('%b %d %Y %H:%M:%S', time.gmtime(time.time()))
data2 = json.dumps(data)

print(data2)

data3 = json.loads(data2)
print(data3["MachineCode"])



from pyDes import des, PAD_PKCS5, ECB
 
#  秘钥  （如果和Java对接，两边要有相同的秘钥）
DES_KEY = "!n@)H:Yd"
 
 # 这里中文要转成字节
 
des_obj = des(DES_KEY, ECB, DES_KEY, padmode=PAD_PKCS5)  # 初始化一个des对象，参数是秘钥，加密方式，偏移， 填充方式
# secret_bytes = des_obj.encrypt(s)   # 用对象的encrypt方法加密
# s = des_obj.decrypt(secret_bytes)   # 用对象的decrypt方法解密
# print(secret_bytes,s)
 
def encrypt(s):
    s = s.encode() # 这里中文要转成字节
    secret_bytes = des_obj.encrypt(s)  # 用对象的encrypt方法加密
    return secret_bytes.hex()
 
def decrypt(secret_bytes):
    secret_bytes = bytes.fromhex(secret_bytes) # 这里中文要转成字节
    s = des_obj.decrypt(secret_bytes)  # 用对象的decrypt方法解密
    return s.decode()
 
test = encrypt(data2)
file = open("./LIC/pyCheck" + '.che' , 'w') 
print(test)
print(decrypt(test))

with open('LICENCE', 'w') as f:

    f.write(test)

with open("LIC/Licence", "r", encoding='UTF-8')as f:
    print(decrypt(f.read()))

BFEBFBFF00090672
14:75:5B:C5:E2:D4,E0:BE:03:7D:30:C6,14:75:5B:C5:E2:D8,14:75:5B:C5:E2:D5,16:75:5B:C5:E2:D4,96:9D:20:52:41:53,98:3C:20:52:41:53,9A:1E:20:52:41:53,00:50:56:C0:00:01,00:50:56:C0:00:08,
{"MachineCode": "d1c6fff94bfa3db1fd2e31604b1fce2e", "StartTime": 1704879948.720736, "FinTime": 2568879948.720736, "Software": "Gyrflacon"}
d1c6fff94bfa3db1fd2e31604b1fce2e
efe0716d56f71bcc6db13bd29a59d465aac6ff7d343e31af24d9de702ba78f45253e51839972ccb8e778a5810be26b2a08311747ab29e3cb37bdbc80c13b1dee9551e172cc2771f354967cc1bb10680ce98ea020993a867c894483f32f221a213828ea742b6af7fc4881f1ab22eb698df7f84fcdad810a00e3d04dab7e740b52c8c806d51a0f67a43b481b180bdb238b
{"MachineCode": "d1c6fff94bfa3db1fd2e31604b1fce2e", "StartTime": 1704879948.720736, "FinTime": 2568879948.720736, "Software": "Gyrflacon"}
{"MachineCode": "fc7ba45834deef6b27488058182fea50", "StartTime": 1704522750.9981394, "FinTime": 2568522750.9981394, "Software": "Gyrflacon"}


In [5]:

from pyDes import des, PAD_PKCS5, ECB
import time
import json
#  秘钥  （如果和Java对接，两边要有相同的秘钥）
DES_KEY = "!n@)H:Yd" 
# 这里中文要转成字节
LICDAY = 10
#462469e2e2f2e63d102ea2bd0f563bf3
MachineCode_REG = 'f2dc1fc17c3b523a3e7dcad2c7d772ee'

Data_REG = { 'MachineCode' : MachineCode_REG, 'StartTime' : time.time(), 'FinTime' : time.time() + 60*60*24*LICDAY , 'Software' : 'Gyrflacon'} 
Data_REG = json.dumps(Data_REG)

des_obj = des(DES_KEY, ECB, DES_KEY, padmode=PAD_PKCS5)  # 初始化一个des对象，参数是秘钥，加密方式，偏移， 填充方式
# secret_bytes = des_obj.encrypt(s)   # 用对象的encrypt方法加密
# s = des_obj.decrypt(secret_bytes)   # 用对象的decrypt方法解密
# print(secret_bytes,s)

def encrypt(s):
    s = s.encode() # 这里中文要转成字节
    secret_bytes = des_obj.encrypt(s)  # 用对象的encrypt方法加密
    return secret_bytes.hex()
 
def decrypt(secret_bytes):
    secret_bytes = bytes.fromhex(secret_bytes) # 这里中文要转成字节
    s = des_obj.decrypt(secret_bytes)  # 用对象的decrypt方法解密
    return s.decode()

LicenseCode = encrypt(Data_REG)
print(LicenseCode)
print(decrypt(LicenseCode))

with open('LICENCE', 'w') as f:

    f.write(LicenseCode)



efe0716d56f71bcc6db13bd29a59d4651478291b0c24b39bcb8ba398fac768bb8599a36b2efd6c98c593521ab7985e6608311747ab29e3cb37bdbc80c13b1deedfed4d2c3bf4e0fdd5cb839ba1ca79f684cd0c95ce3c20d6e204807c7ada8534c379735ed9fc74cad7a3f3bded184b88724fb5f8dbffed3270e27c829b4f52e3f90d12915cdceaf8a224ac91c38aace6
{"MachineCode": "f2dc1fc17c3b523a3e7dcad2c7d772ee", "StartTime": 1704958839.3610291, "FinTime": 1705822839.3610291, "Software": "Gyrflacon"}


In [16]:
import datetime
date = datetime.datetime.fromtimestamp(1705744656) - datetime.datetime.fromtimestamp(1704880626)
print(str(date))

10 days, 0:00:30


In [34]:
import time
import datetime
import hashlib
import os

starttime = time.time()
fintime = time.time()+ 60*60*24*30*1000
datetime.datetime.fromtimestamp(fintime)
#检查时间lic码

def Encrypted(code):
    EncryptStr =  self.get_MachineCode()
    # 使用 DES-CBC加密算法加密机器码
    Des_key = "!n@)H:Yd" 				#自定义 Key，需八位
    Des_IV = "\x11\2\x2a\3\1\x27\2\0"  	# 自定IV向量
    k = des(Des_key, CBC, Des_IV, pad=None, padmode=PAD_PKCS5)
    EncryptStr = k.encrypt(code)
    # 加密结果转 base64 编码
    base64_code = base64.b32encode(EncryptStr)
    # 编码结果使用 MD5 加密
    md5_code = hashlib.md5(base64_code).hexdigest().upper()
    return md5_code


if os.path.exists("./LIC/pyCheck"):
    file = open("./LIC/pyCheck" + '.che' , 'w') 
    pass
else:
    starttime = time.time()
    fintime = time.time()+ 60*60*24*30
    datetime.datetime.fromtimestamp(fintime)
    file = open("./LIC/pyCheck" + '.che' , 'w')
    file.write(Encrypted(code))




NameError: name 'MachineCode' is not defined

In [24]:
import json
import time
starttime = time.time()
data = [ { 'MachineCode' : 'asdv', 'CreatTime' : starttime , 'AdmissionTime' : 60*60*24*10, } ]
text = json.dumps(data )
text

'[{"MachineCode": "asdv", "CreatTime": 1700792089.2572422, "AdmissionTime": 864000}]'

In [31]:

from enum import Enum
class Weekday(Enum):
    monday = 1
    tuesday = 2
    wednesday = 3
    thirsday = 4
    friday = 5
    saturday = 6
    sunday = 7
 
print(Weekday.wednesday.value)


3


In [3]:
class MainPage(QtWidgets.QMainWindow ,  Ui_MainWindow ):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        
    #信息页面
    def OpneSysinfo(self):
        #必须加self，不然不会接入主线程，窗口闪退

        import SystemInfo
        self.page = SystemInfo.SystemPage()
        self.page.show()
        pass
    
    def OpenContactus(self):

        #必须加self，不然不会接入主线程，窗口闪退

        import Contactus

        self.page = Contactus.ContactPage()
        self.page.show()

        pass
        


In [4]:
if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    my_pyqt_form =  MainPage()
    my_pyqt_form.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\qwer1\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
